In [5]:
# !pip install pandas
# !pip install nltk kaggle


import os
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\niloa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\niloa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\niloa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
skills = pd.read_csv("job_skills.csv")
summary = pd.read_csv("job_summary.csv")
title = pd.read_csv("linkedin_job_postings.csv", usecols=['job_link', 'job_title'])

#merge all the datsets
firstdf = pd.merge(skills, summary, on='job_link')
df = pd.merge(firstdf, title, on='job_link')
df

,job_link,job_skills,job_summary,job_title
0,https://www.linkedin.com/jobs/view/housekeeper...,"Building Custodial Services, Cleaning, Janitor...",Department:\nBuilding Custodial Services\nSala...,Housekeeper I - PT
1,https://www.linkedin.com/jobs/view/assistant-g...,"Customer service, Restaurant management, Food ...",Summary Of Key Responsibilities\nThis position...,Assistant General Manager - Huntington 4131
2,https://www.linkedin.com/jobs/view/school-base...,"Applied Behavior Analysis (ABA), Data analysis...",Make a difference every day by joining CCRES a...,School-based Behavior Analyst
3,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Engineering, Project Controls, Sche...",Requisition ID: 271524\nRelocation Authorized:...,Electrical Deputy Engineering Group Supervisor
4,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Assembly, Point to point wiring, St...",Job Description\nProduction Specialist\nElectr...,Electrical Assembly Lead
...,...,...,...,...
1296376,https://www.linkedin.com/jobs/view/community-a...,"Communication Skills, Time Management, Custome...",Job Description\nThe Community Ambassador is a...,Community Ambassador - The Station at Raleigh
1296377,https://www.linkedin.com/jobs/view/sr-it-analy...,"Windows SQL, EDI X12, Edifecs Platform, Health...",Interested in fully remote opportunity as a Sr...,Sr. IT Analyst - Edifecs Technical Systems Ana...
1296378,https://www.linkedin.com/jobs/view/operations-...,"Adaptability, Communication, Digital Fluency, ...",Entity:\nProduction & Operations\nJob Family G...,Operations Excellence Specialist
1296379,https://www.linkedin.com/jobs/view/float-patie...,"CNA, EMT, BLS, Medical Assistant, CPCT, LPN, R...","Overview To be part of our organization, every...",Float Patient Care Associate CSO YNHH


In [7]:
# synonym mapping
synonym_map = {
    "customer service": "customer support",
    "customer support": "customer support",
    "customer success": "customer support",
    "technical support": "customer support",
    "project management": "project manager",
    "project manager": "project manager",
    "product management": "product manager",
    "business analysis": "business analyst",
    "data viz": "data visualization",
    "js": "javascript",
    "java": "java programming",
    "java programming": "java programming",
    "c++": "c plus plus",
    "c plus plus": "c plus plus",
    "c#": "c sharp",
    "c sharp": "c sharp",
    "ruby on rails": "ruby",
    "php": "php programming",
    "php programming": "php programming",
    "html": "html5",
    "html5": "html5",
    "css": "css3",
    "css3": "css3",
    "natural language processing": "nlp",
    "software development": "software engineering",
    "software engineer": "software engineering",
    "web development": "web developer",
    "web engineering": "web developer",
    "data engineering": "data engineering",
    "data engineer": "data engineering",
    "cloud engineering": "cloud computing",
    "python programming": "python",
    "python dev": "python",
    "py": "python",
    "data science": "data analytics",
    "data scientist": "data analytics",
    "data analysis": "data analytics",
    "data analyst": "data analytics",
    "machine learning engineer": "machine learning",
    "ml": "machine learning",
    "ai": "artificial intelligence",
    "artificial intelligence": "artificial intelligence",
    "artificial intelligence engineer": "artificial intelligence",
    "cloud computing": "cloud computing",
    "cloud architect": "cloud computing",
    "cloud developer": "cloud computing",
    "full stack developer": "full stack developer",
    "full stack development": "full stack developer",
    "deep learning engineer": "deep learning",
    "deep learning": "deep learning",
    "big data": "big data",
    "sql": "structured query language",
    "statistics": "statistical analysis",
    "excel": "microsoft office suite",
    "ms excel": "microsoft office suite",
    "powerpoint": "microsoft office suite",
    "ms powerpoint": "microsoft office suite",
    "word": "microsoft office suite",
    "ms word": "microsoft office suite",
        "verbal communication": "communication",
    "written communication": "communication",
    "presentation skills": "communication",
    "public speaking": "communication",
    "teamwork": "collaboration",
    "collaboration skills": "collaboration",
    "interpersonal skills": "collaboration",
    "people skills": "collaboration",
    "time management": "time_management",
    "project management": "project_management",
    "leadership skills": "leadership",
    "decision making": "decision_making",
    "critical thinking": "problem_solving",
    "analytical skills": "problem_solving",
    "negotiation skills": "negotiation",
    "customer service": "customer_service",
    "client relations": "customer_service",
    "account management": "customer_service",
    "sales experience": "sales",
    "business development": "sales",
}

In [8]:
# Function to clean and preprcess the skills data

# Ensure necessary NLTK resources are downloaded
try:
    nltk_stopwords = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    nltk_stopwords = set(stopwords.words('english'))

try:
    lemmatizer = WordNetLemmatizer()
except LookupError:
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    lemmatizer = WordNetLemmatizer()

# Add additional stopwords
additional_stopwords = {"''", "...", "``", "--"}
stop_words = set(nltk_stopwords)
stop_words.update(additional_stopwords)


def preprocess_skills(skills):
    if skills is None:
        return []

    # Ensure skills is string
    skills_str = str(skills) if not isinstance(skills, str) else skills

    # Split by comma and "and"
    skill_list = []
    for skill_item in skills_str.split(','):
        skill_list.extend([part.strip() for part in re.split(r'\band\b', skill_item) if part.strip()])

    cleaned_skills = []
    punctuation = string.punctuation + "–"

    for skill in skill_list:
        # Apply synonym mapping
        skill_lower = skill.lower()
        for key, value in synonym_map.items():
            skill_lower = skill_lower.replace(key, value)

        # Remove punctuation, non-letters, extra spaces
        skill_lower = ''.join([char for char in skill_lower if char not in punctuation])
        skill_lower = re.sub(r'[^a-z\s]', '', skill_lower)
        skill_lower = re.sub(r'\s+', ' ', skill_lower).strip()

        # Tokenize
        tokens = skill_lower.split()

        # Lemmatize and filter stopwords, short words, and generic terms
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        tokens = [
            word for word in tokens
            if word not in stop_words
            and re.match("^[a-zA-Z]+$", word)
            and len(word) > 3
            and word not in {"ability", "skills", "skill"}
        ]

        cleaned_skills.extend(tokens)

    return cleaned_skills


df['cleaned_skills'] = df['job_skills'].apply(preprocess_skills)

# df.head()

In [9]:
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

# Filter rare skills
skill_counts = Counter(skill for skills in df['cleaned_skills'] for skill in skills)
min_freq = 3
df['cleaned_skills'] = df['cleaned_skills'].apply(lambda skills: [s for s in skills if skill_counts[s] >= min_freq])

# Convert cleaned skills back to string for TF-IDF
df['skills_str'] = df['cleaned_skills'].apply(lambda skills: " ".join(skills))
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=3)  
tfidf_matrix = vectorizer.fit_transform(df['skills_str'])

print("TF-IDF matrix shape:", tfidf_matrix.shape)


TF-IDF matrix shape: (1296381, 1244254)


In [ ]:
import pandas as pd
import pickle

# Save the processed DataFrame
# df.to_csv("df_cleaned.csv", index=False)

# with open("/tfidf_matrix.pkl", "wb") as f:
#     pickle.dump(tfidf_matrix, f)

df_cleaned = pd.read_csv("/df_cleaned.csv")
with open("/tfidf_matrix.pkl", "rb") as f:
    tfidf_matrix = pickle.load(f)


In [ ]:
from sklearn.decomposition import TruncatedSVD
import numpy as np

# Dimetion Reduction using TruncatedSVD
svd = TruncatedSVD(n_components=200, random_state=42)
svd.fit(tfidf_matrix)

cumulative_variance = np.cumsum(svd.explained_variance_ratio_)
for i, var in enumerate(cumulative_variance, 1):
    print(f"Components: {i}, Cumulative explained variance: {var:.4f}")

reduced_data = svd.transform(tfidf_matrix)
pd.DataFrame(reduced_data).to_csv("reduced_data.csv", index=False)


Components: 1, Cumulative explained variance: 0.0029
Components: 2, Cumulative explained variance: 0.0090
Components: 3, Cumulative explained variance: 0.0143
Components: 4, Cumulative explained variance: 0.0182
Components: 5, Cumulative explained variance: 0.0216
Components: 6, Cumulative explained variance: 0.0244
Components: 7, Cumulative explained variance: 0.0270
Components: 8, Cumulative explained variance: 0.0293
Components: 9, Cumulative explained variance: 0.0315
Components: 10, Cumulative explained variance: 0.0336
Components: 11, Cumulative explained variance: 0.0355
Components: 12, Cumulative explained variance: 0.0374
Components: 13, Cumulative explained variance: 0.0392
Components: 14, Cumulative explained variance: 0.0409
Components: 15, Cumulative explained variance: 0.0427
Components: 16, Cumulative explained variance: 0.0443
Components: 17, Cumulative explained variance: 0.0459
Components: 18, Cumulative explained variance: 0.0474
Components: 19, Cumulative explained 

In [ ]:
#Even after the reduction, the shape of the data remains large so sampling is necessary

sample_idx = np.random.choice(reduced_data.shape[0], size=int(0.2*reduced_data.shape[0]), replace=False)
reduced_sample = reduced_data[sample_idx]
pd.DataFrame(reduced_sample).to_csv("reduced_sample.csv", index=False)